In [1]:
import librosa
import soundfile as sf
import numpy as np
from fastdtw import fastdtw
import os
import glob
import pandas as pd
from scipy.spatial.distance import euclidean

In [2]:
def noise_reduction(audio, sr, n_fft=512, hop_length=256, n_std_thresh=1.5):
    stft = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
    magnitude, phase = np.abs(stft), np.angle(stft)
    noise_magnitude = np.mean(magnitude[:, :10], axis=1, keepdims=True)
    threshold = noise_magnitude * n_std_thresh
    mask = magnitude > threshold
    stft_clean = mask * magnitude * np.exp(1j * phase)
    audio_clean = librosa.istft(stft_clean, hop_length=hop_length)
    
    return audio_clean

In [3]:
def normalize_audio(filepath, target_rms=0.1):
    audio, sr = librosa.load(filepath, sr=None)
    audio = noise_reduction(audio, sr)
    rms = (audio ** 2).mean() ** 0.5
    scaling_factor = target_rms / rms
    normalized_audio = audio * scaling_factor
    sf.write(filepath, normalized_audio, sr)

In [4]:
def split_audio(file_path, segment_duration=6, sr=16000):
    y, _ = librosa.load(file_path, sr=sr)
    segment_length = segment_duration * sr
    segments = [y[i:i + segment_length] for i in range(0, len(y), segment_length)]
    return segments

In [5]:
def find_files_for_id(base_path, participant_id, keyword):
    
    # search_pattern = os.path.join(base_path, f"ID_{participant_id}_PILOT/ID_{participant_id}_level2_{keyword}.wav")
    # matching_files = glob.glob(search_pattern)

    # return matching_files

    search_pattern_1 = os.path.join(base_path, f"ID_{participant_id}_PILOT/ID_{participant_id}_level2_{keyword}.wav")
    search_pattern_2 = os.path.join(base_path, f"ID_{participant_id}_PILOT/ID_{participant_id}_level2_1_{keyword}.wav")
    
    # Use glob to find matching files for both patterns
    matching_files_1 = glob.glob(search_pattern_1)
    matching_files_2 = glob.glob(search_pattern_2)
    
    # If a file is found in either of the two forms, return the matching files
    if matching_files_1:
        return matching_files_1
    elif matching_files_2:
        return matching_files_2
    

In [6]:
def compute_spectrogram(signal, sr=16000, n_fft=512, hop_length=256):
    spectrogram = librosa.stft(signal, n_fft=n_fft, hop_length=hop_length)
    spectrogram_db = librosa.amplitude_to_db(np.abs(spectrogram), ref=np.max)
    return spectrogram_db.T

In [7]:
def compare_spectrograms(spectrograms1, spectrogram2):
    dtw_distances = []
    for s1 in spectrograms1:
        distance, path = fastdtw(s1, spectrogram2, dist=euclidean)
        dtw_distances.append(distance)
    return dtw_distances

In [8]:
def get_file_paths(keyword, file_paths_dict):
    if keyword in file_paths_dict:
        return file_paths_dict[keyword]
    else:
        return []

In [9]:
pairs = {
    "బట్ట": ("బట్ట"),
    "పట్ట": ("పట్ట"),
    "అట్ట": ("అట్ట"),
    "పాడు": ("పాడు"),
    "పాటు": ("పాటు"),
    "పావు": ( "పావు"),
    "తన్ను": ("తన్ను"),
    "దన్ను": ("దన్ను"),
    "అన్ను": ("అన్ను"),
    "కాటు": ("కాటు"),
    "గాటు": ( "గాటు"),
    "ఆటు": ("ఆటు"),
    "ఫాలు": ("ఫాలు"),
    "వాలు": ("వాలు"),
    "ఆలు": ("ఆలు"),
    "హాయి": ("హాయి"),
    "ఆయి": ("ఆయి")
}
file_paths_dict = {
    "మట్ట": [
        "reference/matta.wav"
    ],
    "బట్ట": [
        "reference/battaa.wav"
    ],
    "పట్ట": [
        "reference/pattaa.wav"
    ],
    "అట్ట": [
        "reference/atta.wav"
    ],
    "పాడు": [
        "reference/paadu.wav"
    ],
    "పాటు": [
        "reference/paatu.wav"
    ],
    "పావు": [
        "reference/paavu.wav"
    ],
    "తన్ను": [
        "reference/thannu.wav"
    ],
    "దన్ను": [
        "reference/dannu.wav"
    ],
    "అన్ను": [
        "reference/annu.wav"
    ],
    "కాటు": [
        "reference/kaatuu.wav"
    ],
    "గాటు": [
        "reference/gaatuu.wav"
    ],
    "ఆటు": [
        "reference/aatu.wav"
    ],
    "ఫాలు": [
        "reference/faalu.wav"
    ],
    "వాలు": [
        "reference/vaalu.wav"
    ],
    "ఆలు": [
        "reference/aalu.wav"
    ],
    "హాయి": [
        "reference/haayi.wav"
    ],
    "ఆయి": [
        "reference/aayi.wav"
    ],
}

In [10]:

# Base directory where files are located
base_directory = "../../Speech-Data/pilot"

# List of participant IDs from 1 to 11
participant_ids = [str(i) for i in range(1, 12)]
participant_ids = [1,2,3,4,5,6,7,9,14,15,17]
# List to accumulate results before creating DataFrame
# Create an empty DataFrame to store results
results_df = pd.DataFrame(columns=["Participant_ID", "Keyword",
                                   "Segment_1_Best_Match", "Segment_1_Min_Distance",
                                   "Segment_2_Best_Match", "Segment_2_Min_Distance",
                                   "Segment_3_Best_Match", "Segment_3_Min_Distance"])

# List to accumulate results before creating DataFrame
results_list = []

# Iterate over each participant ID and keyword pairs
for participant_id in participant_ids:
    print(f"\nFiles for participant ID {participant_id}:")
    
    # Iterate over each keyword in pairs
    for key, keywords in pairs.items():
        # print(f"  Keyword: {key}")
        
        # Find files for the given ID and keyword
        files = find_files_for_id(base_directory, participant_id, key)
        
        if files:
            # Assume we're working with the first file found
            file = files[0]
            
            a_segments = split_audio(file)
            a_spectrograms = [compute_spectrogram(segment) for segment in a_segments]
            
            ref_spectograms = []
            for k in keywords:
                ref_files = get_file_paths(k, file_paths_dict)
                for ref in ref_files:
                    # normalize_audio(ref)
                    ref_y, _ = librosa.load(ref, sr=16000)
                    ref_spectrogram = compute_spectrogram(ref_y)
                    ref_spectograms.append(ref_spectrogram)
            
            # Initialize variables to track matches for each segment
            segment_matches = []

            # Compare spectrograms of all segments
            for idx, spectrogram in enumerate(a_spectrograms):
                a_vs_b_similarities_dtw_spec = compare_spectrograms(ref_spectograms, spectrogram)
                min_distance = min(a_vs_b_similarities_dtw_spec)
                best_match = keywords[a_vs_b_similarities_dtw_spec.index(min_distance)]
                
                # Store the best match and distance for each segment
                segment_matches.append((best_match, min_distance))

            # Ensure we have results for exactly 3 segments
            while len(segment_matches) < 3:
                segment_matches.append(("", float('inf')))
                
            # Accumulate the result in a list
            results_list.append({
                "Participant_ID": participant_id,
                "Keyword": key,
                "Segment_1_Best_Match": segment_matches[0][0],
                "Segment_1_Min_Distance": segment_matches[0][1],
                "Segment_2_Best_Match": segment_matches[1][0],
                "Segment_2_Min_Distance": segment_matches[1][1],
                "Segment_3_Best_Match": segment_matches[2][0],
                "Segment_3_Min_Distance": segment_matches[2][1]
            })

        else:
            print(f"No files found for {key}")

# Convert the list of results to a DataFrame
results_df = pd.DataFrame(results_list)



Files for participant ID 1:


ValueError: min() arg is an empty sequence

In [ ]:
# Save the DataFrame to a CSV file with UTF-8 encoding
csv_file_path = "comparison_results_SELFMatch.csv"
results_df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

In [ ]:
print("\nComparison Results:")
results_df


Comparison Results:


,Participant_ID,Keyword,Segment_1_Best_Match,Segment_1_Min_Distance,Segment_2_Best_Match,Segment_2_Min_Distance,Segment_3_Best_Match,Segment_3_Min_Distance
0,1,బట్ట,బట్ట,26876.508435,బట్ట,32514.006408,బట్ట,27071.609543
1,1,పట్ట,బట్ట,26728.107306,బట్ట,25302.239170,బట్ట,20597.251352
2,1,పాడు,పాడు,27153.322659,పాడు,30574.545974,పాడు,27009.280378
3,1,పాటు,పాడు,26252.911984,పాడు,25117.796288,పాడు,22412.020545
4,1,తన్ను,తన్ను,34825.561052,తన్ను,37048.458412,తన్ను,34465.000439
...,...,...,...,...,...,...,...,...
89,17,దన్ను,తన్ను,34876.103476,తన్ను,24953.635262,తన్ను,24439.352421
90,17,కాటు,కాటు,21779.572892,కాటు,14082.147878,కాటు,12026.393308
91,17,గాటు,కాటు,29741.120228,కాటు,21244.118831,కాటు,15508.218428
92,17,ఫాలు,ఫాలు,39346.098911,వాలు,26643.289359,ఫాలు,24902.718965
